<a href="https://colab.research.google.com/github/cestmhd/TensorflowPractice/blob/main/CNN_color_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import matplotlib.pyplot as plt


In [3]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [5]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
zip_dir = tf.keras.utils.get_file('cats_and_dogs_filterted.zip', origin=_URL, extract=True)

In [7]:
zip_dir_base = os.path.dirname(zip_dir)
print(zip_dir_base)

/root/.keras/datasets


In [8]:
base_dir = os.path.join(os.path.dirname(zip_dir), 'cats_and_dogs_filtered')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')  # directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')  # directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')  # directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')  # directory with our validation dog pictures

In [9]:
num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))

num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val

In [10]:
print('total training cat images:', num_cats_tr)
print('total training dog images:', num_dogs_tr)

print('total validation cat images:', num_cats_val)
print('total validation dog images:', num_dogs_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

total training cat images: 1000
total training dog images: 1000
total validation cat images: 500
total validation dog images: 500
--
Total training images: 2000
Total validation images: 1000


In [11]:
batch_size =100
image_size = 150

In [13]:
train_image_generator = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [14]:
train_image_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                            directory=train_dir,
                                                            shuffle=True,
                                                            target_size=(image_size,image_size), #(150,150)
                                                            class_mode='binary')
validation_image_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                                      directory = validation_dir,
                                                                      shuffle=False,
                                                                      target_size = (image_size,image_size),
                                                                      class_mode = 'binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [15]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2)
])

In [16]:
model.compile(optimizer = 'Adam',
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

In [18]:
epochs = 100
model.fit_generator(train_image_gen,
                    epochs = epochs,
                    steps_per_epoch = np.ceil(total_train/batch_size),
                    validation_data=validation_image_gen,
                    validation_steps = np.ceil(total_val/batch_size)
                    )

<ipython-input-18-8788a268f0ce>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_image_gen,


Epoch 1/100
20/20 [==============================] - 17s 429ms/step - loss: 0.7133 - accuracy: 0.5110 - val_loss: 0.6927 - val_accuracy: 0.5000
Epoch 2/100
20/20 [==============================] - 8s 415ms/step - loss: 0.6925 - accuracy: 0.4920 - val_loss: 0.6865 - val_accuracy: 0.5620
Epoch 3/100
20/20 [==============================] - 8s 415ms/step - loss: 0.6814 - accuracy: 0.5785 - val_loss: 0.6752 - val_accuracy: 0.5810
Epoch 4/100
20/20 [==============================] - 8s 419ms/step - loss: 0.6652 - accuracy: 0.5850 - val_loss: 0.6402 - val_accuracy: 0.6050
Epoch 5/100
20/20 [==============================] - 8s 416ms/step - loss: 0.6474 - accuracy: 0.6120 - val_loss: 0.6346 - val_accuracy: 0.6240
Epoch 6/100
20/20 [==============================] - 9s 463ms/step - loss: 0.6015 - accuracy: 0.6660 - val_loss: 0.6122 - val_accuracy: 0.6590
Epoch 7/100
20/20 [==============================] - 8s 416ms/step - loss: 0.5487 - accuracy: 0.7175 - val_loss: 0.6603 - val_accuracy: 0.628